Tunning de hiperparámetros (aplicación en MNIST)

In [ ]:
pip install tensorflow scikeras

In [7]:
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [10]:
# Cargamos MNIST
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [11]:
# Preprocesamiento de datos
x_train = x_train.reshape(x_train.shape[0], 784).astype('float32') / 255
x_test = x_test.reshape(x_test.shape[0], 784).astype('float32') / 255

In [ ]:
# PARTE 1: Búsqueda de mejores epochs, red simple, una capa oculta, y se entrena el modelo segun la cantidad de epochs.
def crear_modelo():
    modelo = Sequential()
    modelo.add(Dense(units=512, activation='relu', input_shape=(784,)))
    modelo.add(Dense(10, activation='softmax'))
    # Compile model
    modelo.compile(loss='sparse_categorical_crossentropy',  # Cambiado para labels enteros
                  optimizer='adam', 
                  metrics=['accuracy'])
    return modelo

# Configuramos semilla para reproducibilidad
seed = 7
tf.random.set_seed(seed)

# Creamos el wrapper de Keras para sklearn
model = KerasClassifier(model=crear_modelo, verbose=0)

# Definimos grid de hiperparámetros para epochs
param_grid = dict(epochs=[10, 20, 30])

# Ejecutar GridSearchCV
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(x_train, y_train)

# Mostramos resultados
print("=== Resultados búsqueda de EPOCHS ===")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

print(f"\nMejor: {grid_result.best_score_} usando {grid_result.best_params_}")


In [ ]:
# PARTE 2: Búsqueda de batch_size y epochs
param_grid2 = dict(batch_size=[10, 20], epochs=[3, 5])

grid2 = GridSearchCV(estimator=model, param_grid=param_grid2, n_jobs=-1, cv=3)
grid_result2 = grid2.fit(x_train, y_train)

print("\n=== Resultados búsqueda de BATCH_SIZE y EPOCHS ===")
means = grid_result2.cv_results_['mean_test_score']
stds = grid_result2.cv_results_['std_test_score']
params = grid_result2.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

print(f"\nMejor: {grid_result2.best_score_} usando {grid_result2.best_params_}")

In [ ]:
# PARTE 3: Búsqueda de número de neuronas (1 capa oculta)
def create_model_2(neurons1=512):
    """Modelo con número variable de neuronas en primera capa"""
    model = Sequential()
    model.add(Dense(units=neurons1, activation='relu', input_shape=(784,)))
    model.add(Dense(10, activation='softmax'))
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy', 
                  optimizer='adam', 
                  metrics=['accuracy'])
    return model

model2 = KerasClassifier(model=create_model_2, verbose=0)
param_grid3 = dict(model__neurons1=[50, 100])

grid_3 = GridSearchCV(estimator=model2, param_grid=param_grid3, n_jobs=-1, cv=3, scoring='accuracy')
grid_result3 = grid_3.fit(x_train, y_train, epochs=5)

print("\n=== Resultados búsqueda de NEURONAS (1 capa) ===")
means = grid_result3.cv_results_['mean_test_score']
stds = grid_result3.cv_results_['std_test_score']
params = grid_result3.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

print(f"\nMejor: {grid_result3.best_score_} usando {grid_result3.best_params_}")

In [ ]:
# PARTE 4: Búsqueda con 2 capas ocultas (opcional/extendido)
def create_model_3(neurons_1=256, neurons_2=128):
    """Modelo con 2 capas ocultas variables"""
    model = Sequential()
    model.add(Dense(units=neurons_1, activation='relu', input_shape=(784,)))
    model.add(Dense(units=neurons_2, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy', 
                  optimizer='adam', 
                  metrics=['accuracy'])
    return model

model3 = KerasClassifier(model=create_model_3, verbose=0)
param_grid4 = dict(
    model__neurons_1=[128, 256],
    model__neurons_2=[64, 128]
)

grid_4 = GridSearchCV(estimator=model3, param_grid=param_grid4, n_jobs=-1, cv=3, scoring='accuracy')
grid_result4 = grid_4.fit(x_train, y_train, epochs=5)

print("\n=== Resultados búsqueda de NEURONAS (2 capas) ===")
means = grid_result4.cv_results_['mean_test_score']
stds = grid_result4.cv_results_['std_test_score']
params = grid_result4.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

print(f"\nMejor: {grid_result4.best_score_} usando {grid_result4.best_params_}")

In [ ]:
print("\n=== EVALUACIÓN FINAL ===")
best_model = grid_result4.best_estimator_
test_score = best_model.score(x_test, y_test)
print(f"Accuracy en test set: {test_score:.4f}")